# Download Airnow Reporting Area Data

Downloads Airnow's Reporting Area data files (`reportingarea.dat`), saving them in `airnow-data/reporting-area/dat`.  Airnow says the file is updated sub-hourly, at :25 and :40 past the hour.  This script runs once per hour, at :45 past the hour.

Reports to stat.createlab.org as `Airnow Reporting Area Data - Download`.

Docs for the data files are here: https://s3-us-west-1.amazonaws.com//files.airnowtech.org/airnow/docs/ReportingAreaFactSheet.pdf

In [ ]:
import json, os, dateutil, re, requests, subprocess, datetime, glob, stat

from dateutil import rrule, tz, parser

In [ ]:
# Boilerplate to load utils.ipynb
# See https://github.com/CMU-CREATE-Lab/python-utils/blob/master/utils.ipynb

def exec_ipynb(filename_or_url):
    nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
    if(nb['nbformat'] >= 4):
        src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
    else:
        src = [''.join(cell['input']) for cell in nb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']

    tmpname = '/tmp/%s-%s-%d.py' % (os.path.basename(filename_or_url),
                                    datetime.datetime.now().strftime('%Y%m%d%H%M%S%f'),
                                    os.getpid())
    src = '\n\n\n'.join(src)
    open(tmpname, 'w').write(src)
    code = compile(src, tmpname, 'exec')
    exec(code, globals())

exec_ipynb('./python-utils/utils.ipynb')
exec_ipynb('./airnow-common.ipynb')

In [ ]:
EARLIEST_DATA_FILE_DATETIME = datetime.datetime(2020, 1, 1)

SECONDS_TO_PAUSE_BETWEEN_DOWNLOADS = 0.5

NUM_TRAILING_DAYS = 5

RUN_INTERVAL_SECONDS = 60 * 60    # every hour

DAT_DIRECTORY = AirnowCommon.REPORTING_AREA_DAT_DIRECTORY

STAT_SERVICE_NAME = 'Airnow Reporting Area Data - Download'
STAT_HOSTNAME = 'hal21'
STAT_SHORTNAME = 'airnow-reporting-area-data-download'

In [ ]:
Stat.set_service(STAT_SERVICE_NAME)

In [ ]:
# Mirrors the file for the specified timestamp and, if new, returns the path to the file; otherwise returns None
def mirror_timestamp(timestamp):
    filename = timestamp.strftime('%Y%m%d.dat')
    src = AirnowCommon.directory_from_date(timestamp) + '/reportingarea.dat'
    dest = DAT_DIRECTORY + '/' + filename

    (is_new, message, status_code, new_records_file) = AirnowCommon.mirror_airnow_file_and_extract_new_records(src, dest)

    if is_new:
        Stat.info(message, host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
        return dest, new_records_file
    else:
        if status_code == 304:
            print(message) # simply do a print here to reduce noise sent to stat.createlab.org
            # Stat.info(message, host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
        elif status_code < 400:
            Stat.info(message, host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
        else:
            Stat.warning(message, host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)

        return None, None

#mirror_timestamp(dateutil.parser.parse('2021-07-30 00:00'))

In [ ]:
def compute_first_date_to_check():
    files = glob.glob(DAT_DIRECTORY + '/[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9].dat') # YYYYMMDD.dat
    if len(files) == 0:
        return EARLIEST_DATA_FILE_DATETIME
    last_file = sorted(files)[-1]
    last_date = datetime.datetime.strptime(last_file, DAT_DIRECTORY + "/%Y%m%d.dat")
    Stat.debug('Most recently mirrored data file is %s (%s)' % (last_file, last_date), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    first_date_to_check = last_date - datetime.timedelta(days=NUM_TRAILING_DAYS)
    Stat.debug('Checking for updates starting with date %s' % (first_date_to_check), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    sys.stdout.flush()
    return first_date_to_check

# compute_first_date_to_check()

In [ ]:
def mirror():
    starting_timestamp = datetime.datetime.now().timestamp()
    start = compute_first_date_to_check()
    now = datetime.datetime.utcnow()
    timestamps_to_mirror = list(rrule.rrule(rrule.DAILY, dtstart=start, until=now))

    Stat.info('Mirroring %d data files, starting with %s... (up-to-date files will not be logged here)' % (len(timestamps_to_mirror), start), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    for timestamp in timestamps_to_mirror:
        (mirrored_file, new_records_file) = mirror_timestamp(timestamp)
        time.sleep(SECONDS_TO_PAUSE_BETWEEN_DOWNLOADS)

    elapsed_seconds = datetime.datetime.now().timestamp() - starting_timestamp
    Stat.up('Done mirroring %d data files (elapsed time: %d seconds)' % (len(timestamps_to_mirror), elapsed_seconds), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME, valid_for_secs=RUN_INTERVAL_SECONDS*1.5)

def mirror_forever():
    while True:
        mirror()
        sleep_until_next_period(RUN_INTERVAL_SECONDS, 45*60) # start at 45 minutes after the hour

mirror_forever()